# Formatting Numbers

This notebook explores the default behavior of number formatting and demonstrates how you can customize it.

Refer to the [Formatting](https://lets-plot.org/python/pages/formats.html) documentation page for details on supported format strings and string templates.

The general formatting system works as follows:

- **Axis scales** determine the format for legends, breaks, and tooltips.
- **Text geometries** define the format for layers and tooltips.
- **Tooltip formatting** can always be explicitly specified.

In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/refs/heads/master/data/chemical_elements.csv", encoding_errors='ignore')
print(df.shape)
df.head()

(118, 23)


,Atomic Number,Element,Symbol,Atomic Weight,Period,Group,Phase,Most Stable Crystal,Type,Ionic Radius,...,Density,Melting Point (K),Boiling Point (K),Isotopes,Discoverer,Year of Discovery,Specific Heat Capacity,Electron Configuration,Display Row,Display Column
0,1,Hydrogen,H,1.007940,1,1,gas,NaN,Nonmetal,0.012,...,0.000090,14.175,20.28,3.0,Cavendish,1766.0,14.304,1s1,1,1
1,2,Helium,He,4.002602,1,18,gas,NaN,Noble Gas,NaN,...,0.000179,NaN,4.22,5.0,Janssen,1868.0,5.193,1s2,1,18
2,3,Lithium,Li,6.941000,2,1,solid,bcc,Alkali Metal,0.760,...,0.534000,453.850,1615.00,5.0,Arfvedson,1817.0,3.582,[He] 2s1,2,1
3,4,Beryllium,Be,9.012182,2,2,solid,hex,Alkaline Earth Metal,0.350,...,1.850000,1560.150,2742.00,6.0,Vaulquelin,1798.0,1.825,[He] 2s2,2,2
4,5,Boron,B,10.811000,2,13,solid,rho,Metalloid,0.230,...,2.340000,2573.150,4200.00,6.0,Gay-Lussac,1808.0,1.026,[He] 2s2 2p1,2,13


## Default Number Formatting

By default, all numeric aesthetics are formatted using the `'g'` type, with commas for thousands separators and truncated trailing zeros. For more details, see the [Formatting](https://lets-plot.org/python/pages/formats.html) documentation page.

In [4]:
(ggplot(df)
 + geom_point(aes("Melting Point (K)", "Boiling Point (K)", size="Atomic Radius", color="Type"),
              tooltips=layer_tooltips(["Type", "Atomic Radius"]).title("@Symbol\n@Element"))
 + scale_color_discrete(guide='none'))

## Scale Format

Sometimes, the default formatting may not be ideal.
For instance, when displaying years, you might prefer to remove the comma.

In [5]:
(ggplot(df)
 + geom_lollipop(aes("Year of Discovery", "Atomic Number", color="Type", size="Atomic Radius"),
                 tooltips=layer_tooltips(["Type", "Atomic Number", "Atomic Radius", "Year of Discovery"])
                         .title("@Symbol\n@Element"))
 + scale_color_discrete(guide='none')
 + scale_size(range=[1, 3])
 + ggsize(1000, 400))

To remove the comma in years, use the `'d'` format and apply it to the scale to update the display across all chart elements (axes, tooltips, legends).

In [6]:
(ggplot(df)
 + geom_lollipop(aes("Year of Discovery", "Atomic Number", color="Type", size="Atomic Radius"),
                 tooltips=layer_tooltips(["Type", "Atomic Number", "Atomic Radius", "Year of Discovery"])
                         .title("@Symbol\n@Element"))
 + scale_x_continuous(format="d") # Set custom format via scale parameter
 + scale_color_discrete(guide='none')
 + scale_size(range=[1, 3])
 + ggsize(1000, 400))

To modify the format of the atomic radius in the legend, use the `format` parameter within `scale_size()`. For tooltips, adjust the `tooltips` parameter in the geometry.

In [7]:
(ggplot(df)
 + geom_lollipop(aes("Year of Discovery", "Atomic Number", color="Type", size="Atomic Radius"),
                 tooltips=layer_tooltips(["Type", "Atomic Number", "Atomic Radius", "Year of Discovery"])
                         .title("@Symbol\n@Element")
                         .format("@{Atomic Radius}", """{.2f}\(\cdot 10^{-10}}\) meters""")) # Set custom format via layer_tooltips() class
 + scale_x_continuous(format="d")
 + scale_color_discrete(guide='none')
 + scale_size(range=[1, 3], format="""{.2f}\(\cdot 10^{-10}}\) meters""")                    # Set custom format via scale
 + ggsize(1000, 400))

## Formatting for Aesthetics Without Scale

### Positional Aesthetics

"Additional" aesthetics, such as `ymin`, `ymax`, `lower`, `middle`, `upper`, and others, follow the scale format of the corresponding "main" aesthetics (`x` or `y`).

In [8]:
gggrid([
    (ggplot(df)
     + geom_boxplot(aes("Type", "Atomic Radius"))
     + ggtitle("Default")),
    (ggplot(df)
     + geom_boxplot(aes("Type", "Atomic Radius"))
     + scale_y_continuous(format=".3f")
     + ggtitle("Custom y scale format"))
])

### Other Aesthetics

Some aesthetics, such as `width`, `violinwidth`, `quantile`, `angle`, and others, do not have a corresponding `scale_xxx()` function. These values only appear in tooltips and are formatted exclusively through the `tooltips` parameter of a geometry.

An exception to this rule is the `label` aesthetic, which can be customized using the `label_format` parameter in the geometry.

In the table below, the atomic weight format `"[{.1f}]"` is applied to the label using the `label_format` parameter. This format is also inherited by the tooltip.

In [9]:
(ggplot(df)
 + geom_tile(aes("Display Column", "Display Row", fill="Type"),
             width=.9, height=.9, alpha=.5, tooltips='none')
 + geom_text(aes("Display Column", "Display Row", label="Symbol"),
             nudge_y=.2, size=9)
 + geom_text(aes("Display Column", "Display Row", label="Atomic Weight"),
             nudge_y=-.2, size=5,
             tooltips=layer_tooltips(["Type", "Atomic Weight"]).title("@Symbol\n@Element"),
             label_format="[{.1f}]") # Use label_format to change the default in labels and tooltips
 + scale_y_reverse()
 + theme_void()
 + ggsize(1000, 400))

## Exponent Format

Formatting is also influenced by the `exponent_format` parameter in the `theme()` function. This parameter enhances all formatting applied to the plot.

In the plot below, "power" notation is enabled for the axes.

In [10]:
(ggplot(df)
 + geom_point(aes("Melting Point (K)", "Boiling Point (K)", size="Atomic Radius", color="Type"),
              tooltips=layer_tooltips(["Type", "Atomic Radius"]).title("@Symbol\n@Element"))
 + scale_color_discrete(guide='none')
 + theme(exponent_format=('pow', None, 2))) # Look at the axes

## Notes

### Annotation Formatting Is Like Tooltip Formatting

In [11]:
group_df = df.groupby("Type")["Atomic Weight"].mean().to_frame().reset_index()

In [12]:
(ggplot(group_df)
 + geom_pie(aes(fill=as_discrete("Type", order_by="..count.."), weight="Atomic Weight"),
            size=50, show_legend=False, tooltips='none',
            labels=layer_labels(["Type", "Atomic Weight"])
                  .size(10)
                  .format("@{Atomic Weight}", "[{.2~f}]")) # Custom annotation formatting
 + theme_void()
 + ggsize(800, 600))

### Formatting Is Not Bound to Data Variables

Even when two aesthetics reference the same variable from a dataset, their formatting may differ.
To ensure consistent formatting, adjustments must be made manually.

In the example below, both the `y` and `label` aesthetics reference the "Atomic Weight" variable. However, the text label on the plot and the tooltip are formatted differently.

In [13]:
metalloid_df = df[df["Type"] == "Metalloid"]

In [14]:
(ggplot(metalloid_df, aes("Melting Point (K)", "Atomic Weight"))
 + geom_text(aes(label="Symbol"), nudge_y=5, size=10,
             tooltips=layer_tooltips(["Melting Point (K)", "Atomic Weight"]) # y aes mapped to "Atomic Weight"
                     .title("@Symbol\n@Element"))
 + geom_text(aes(label="Atomic Weight"), nudge_y=-5, size=5))                # label aes mapped to "Atomic Weight"

This can be resolved by explicitly applying the `".2~f"` format:

In [15]:
(ggplot(metalloid_df, aes("Melting Point (K)", "Atomic Weight"))
 + geom_text(aes(label="Symbol"), nudge_y=5, size=10,
             tooltips=layer_tooltips(["Melting Point (K)", "Atomic Weight"])
                     .format("@{Atomic Weight}", ".2~f")                           # Set custom format in tooltips
                     .title("@Symbol\n@Element"))
 + geom_text(aes(label="Atomic Weight"), nudge_y=-5, size=5, label_format=".2~f")) # Set custom format in labels

### Infinitesimal Density Values

Since the default formatting type is `'g'`, extremely small non-zero values are never rounded to zero. This behavior can be observed in the tooltips for heights in the plot below.

In [16]:
(ggplot(df)
 + geom_area_ridges(aes("Atomic Weight", "Type", fill="Type"))
 + ggsize(1000, 400))

This can be fixed by using the `'f'`-type format, such as `".3~f"`:

In [17]:
(ggplot(df)
 + geom_area_ridges(aes("Atomic Weight", "Type", fill="Type"),
                    tooltips=layer_tooltips().format("^height", ".3~f")) # Fix default formatting
 + ggsize(1000, 400))